In [1]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
buy_times = 0
max_profit = 0
min_loss = 0

C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [3]:
symbols = """
select distinct(name) from stockdataweekly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [4]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [7]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        
sql1= """
select distinct(name) from stockdataweekly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)

<h1> Updating resources

In [9]:
from tqdm import tqdm

pbar = tqdm(total= len(symbol_list))

for symbol in symbol_list:
    symbol_data = """
    select * from stockdataweekly where name = '%s'
    """ % symbol
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)
    pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████▉| 3868/3870 [02:14<00:00, 31.14it/s]

In [10]:
sql_dates = """
select distinct(date) from stockdataweekly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

In [11]:
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

<h1> Loading ANN Model

In [12]:
model = tf.keras.models.load_model('Weekly69-66ANN.model', custom_objects={'sensitivity': sensitivity})

<h1> Trading strategy with the training sets generation

In [13]:

def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global min_loss 
    global max_profit
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdataweekly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    
                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        training_inst = training_set_extractor(data_till_date, date)
                        for_predict = np.asarray(tf.keras.utils.normalize([[training_inst[0], training_inst[1]]]))                          
                        if(model.predict(for_predict) > 0.5):
                            stop_order = current_symbol.support * 0.97  
                            quantity = (0.02 * remaining_cash) // (current_symbol.current_price - stop_order)
                            if quantity > 1:
                                current_symbol.bought = current_price
                                remaining_cash = remaining_cash - commission - (quantity * current_price)                        
                                position[symbol] = quantity
                                current_symbol.times_bought += 1
                                print("Buying ", quantity, "stocks of ", symbol, "At price", current_symbol.bought, "on date", date)                                      
                                buy_times += 1
                                current_symbol.stop_order = stop_order  

                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                
                                win += 1
                                current_symbol.flag_for_wins = True
                        
                        
                    if current_symbol.stop_order > current_price:
                        
                        
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            
                            if current_symbol.flag_for_wins == False:
                                win += 1
                            current_symbol.flag_for_wins = False
                            
                        else:
                            loss += 1
                            
                            current_symbol.flag_for_wins = False
                            
                             
                        if profit < min_loss:
                            min_loss = profit
                        if profit > max_profit:
                            max_profit = profit
                            
                        profit += profit_today
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("\n")                    

<h1> Helper functions for the strategy

In [14]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    past_ten_weeks = data.tail(10)
    difference = abs(len(past_ten_weeks['2. high'].tolist()) - 10)
    zeros_list = [0] * (difference)
    training_instance = [zeros_list[:], zeros_list[:], 'na']
        
    training_instance[0].extend(past_ten_weeks['2. high'].tolist()[:])
    training_instance[1].extend(past_ten_weeks['5. volume'].tolist()[:])
    return training_instance[:]
   

In [15]:
automatic_trader(symbol_list)

Buying  288.0 stocks of  AAME At price 2.875 on date 2000-01-28
Buying  21.0 stocks of  AWRE At price 38.0 on date 2000-01-28
Buying  37.0 stocks of  BKH At price 25.0 on date 2000-01-28
Buying  53.0 stocks of  BOSC At price 9.06 on date 2000-01-28
Buying  22.0 stocks of  CACI At price 26.5 on date 2000-01-28
Buying  55.0 stocks of  CHH At price 17.38 on date 2000-01-28
Buying  199.0 stocks of  CMU At price 5.875 on date 2000-01-28
Buying  113.0 stocks of  DNR At price 4.563 on date 2000-01-28
Buying  6.0 stocks of  DTE At price 41.31 on date 2000-01-28
Buying  22.0 stocks of  EGN At price 18.94 on date 2000-01-28
Buying  3.0 stocks of  FORR At price 64.38 on date 2000-01-28
Buying  51.0 stocks of  GUT At price 8.438 on date 2000-01-28
Buying  21.0 stocks of  HE At price 30.5 on date 2000-01-28
Buying  9.0 stocks of  HLX At price 34.38 on date 2000-01-28
Buying  17.0 stocks of  ICAD At price 3.81 on date 2000-01-28
Buying  2.0 stocks of  IIVI At price 28.0 on date 2000-01-28
Buying  6.

100%|██████████████████████████████████████████████████████████████████████████████| 3870/3870 [02:30<00:00, 31.14it/s]

Buying  6.0 stocks of  NHTC At price 1.75 on date 2000-02-25
Buying  2.0 stocks of  PHIIK At price 12.0 on date 2000-02-25
Buying  3.0 stocks of  NUV At price 8.188 on date 2000-03-03
Buying  2.0 stocks of  OFC At price 8.12 on date 2000-03-03
Buying  2.0 stocks of  VCF At price 12.44 on date 2000-03-03
Buying  2.0 stocks of  YUMA At price 0.812 on date 2000-03-10
Buying  6.0 stocks of  AGX At price 0.69 on date 2000-03-17
Buying  2.0 stocks of  TGA At price 0.7 on date 2000-03-17
Selling  55.0 stocks of CHH  at price 15.0 on date 2000-03-24
Buying  2.0 stocks of  COKE At price 53.0 on date 2000-03-24
Buying  5.0 stocks of  CPT At price 27.75 on date 2000-03-24
Buying  2.0 stocks of  CR At price 23.69 on date 2000-03-24
Buying  4.0 stocks of  CUO At price 23.5 on date 2000-03-24
Buying  10.0 stocks of  DAVE At price 2.688 on date 2000-03-24
Buying  2.0 stocks of  DHI At price 14.0 on date 2000-03-24
Buying  2.0 stocks of  EE At price 10.44 on date 2000-03-24
Buying  3.0 stocks of  EQS 

Selling  53.0 stocks of BOSC  at price 0.47 on date 2003-02-21
Buying  2.0 stocks of  CHN At price 14.33 on date 2003-02-21
Buying  5.0 stocks of  DAR At price 2.14 on date 2003-02-21
Buying  2.0 stocks of  ESE At price 20.94 on date 2003-02-21
Buying  5.0 stocks of  EVY At price 13.8 on date 2003-02-21
Buying  5.0 stocks of  FIZZ At price 10.95 on date 2003-02-21
Buying  5.0 stocks of  GSS At price 1.82 on date 2003-02-21
Buying  3.0 stocks of  HIFS At price 19.5 on date 2003-02-21
Buying  2.0 stocks of  MFL At price 12.5 on date 2003-02-21
Buying  2.0 stocks of  MUH At price 12.06 on date 2003-02-21
Buying  3.0 stocks of  NEA At price 15.09 on date 2003-02-21
Buying  2.0 stocks of  IPCC At price 16.76 on date 2003-03-07
Buying  2.0 stocks of  UBP At price 13.4 on date 2003-05-16
Buying  2.0 stocks of  APWC At price 2.5 on date 2003-09-19
Buying  2.0 stocks of  OSBCP At price 11.2 on date 2003-10-10
Buying  2.0 stocks of  SQBG At price 0.6484 on date 2003-10-10
Buying  3.0 stocks of  

Buying  2.0 stocks of  LIQT At price 3.5 on date 2011-09-23
Selling  2.0 stocks of SNFCA  at price 1.44 on date 2011-10-07
Selling  5.0 stocks of CCCL  at price 3.0 on date 2011-10-14
Selling  5.0 stocks of CO  at price 2.65 on date 2011-10-28
Selling  2.0 stocks of MDC  at price 21.73 on date 2012-01-27
Buying  3.0 stocks of  ENT At price 9.66 on date 2012-02-10
Selling  4.0 stocks of SGOC  at price 0.98 on date 2012-03-23
Selling  3.0 stocks of TAT  at price 1.33 on date 2012-03-23
Buying  2.0 stocks of  TGLS At price 9.85 on date 2012-10-12
Selling  2.0 stocks of ABEV  at price 34.33 on date 2012-10-19
Buying  2.0 stocks of  BBRC At price 20.95 on date 2012-10-19
Selling  2.0 stocks of FTNT  at price 19.89 on date 2012-10-26
Buying  2.0 stocks of  BSCK At price 21.44 on date 2012-11-02
Selling  3.0 stocks of HIFS  at price 63.1 on date 2012-11-23
Buying  2.0 stocks of  ICB At price 19.49 on date 2012-11-23
Buying  2.0 stocks of  KAP At price 25.5 on date 2012-11-23
Selling  3.0 stoc

In [16]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

Maximum Profit is  3686.788999999999
Minimum Loss is  -3097.0666
Profit 3686.748999999999
Remaining cash  12436.749000000007
Win Percentage
55.72755417956656
